# Network building

Based on artists sharing a common announcement/exhibition: using *announcements.json* / *announcements_contemporary.json* 

## Initial steps

In [1]:
import json
with open('data/artists_cleaned_v1.txt', 'r', encoding='utf-8') as f:
    artists = f.read().splitlines()

with open('data/announcements.json', 'r', encoding='utf-8') as f:
    announcements = json.load(f)

## 1) Network building

Something like:

- Add all artists (from the txt file) to a node in the graph
- Run through all announcements, add edges between artists that share an announcement

- Graph should be undirected + weighted, weight = number of announcements shared

In [2]:
import graph_tool as gt
from graph_tool import inference

In [ ]:
g = gt.Graph(directed=False)

# Create a dictionary to map artist names to vertices
artist_to_vertex = {}

for announcement in announcements.values():
    announcement_artists = announcement['artists']
    for artist in artists:
        # If the artist is not already in the graph, add them
        if artist not in artist_to_vertex:
            v = g.add_vertex()
            artist_to_vertex[artist] = v

Make an artist name from vertex dict(/flip out artist_to_vertex dictionary) so we can look up artists by vertex's property 

In [ ]:
artist_name = g.new_vertex_property("string") #vertex_to_artist = {v: artist for artist, v in artist_to_vertex.items()}
for v, artist in artist_to_vertex.items():
    artist_name[v] = artist

g.vertex_properties["artist_name"] = artist_name

Add edges:

In [ ]:
edge_weights = {}
for announcement in announcements.values():
    artists_ = announcement['artists']
    for artist1 in artists_: #Could also just use itertools.combinations
        for artist2 in artists_:
            if artist1 != artist2:
                #First, check if the edge already exists (this returns the edge, or None)
                edge = g.edge(artist_to_vertex[artist1], artist_to_vertex[artist2])
                if edge:
                    edge_weights[edge] += 1
                else:
                    edge = g.add_edge(artist_to_vertex[artist1], artist_to_vertex[artist2])
                    edge_weights[edge] = 1

Add edge weight property to the edges

In [ ]:
weight = g.new_edge_property("int")
for edge, weight_value in edge_weights.items():
    weight[edge] = weight_value
g.edge_properties["weight"] = weight